In [31]:
import requests
from bs4 import BeautifulSoup
import asyncio
import nest_asyncio
nest_asyncio.apply()
import aiohttp
from urllib.parse import urlparse, urljoin, urlsplit

In [32]:
print(urlsplit("http://bit.edu.cn"))
print(urlparse("http://www.bit.edu.cn/docs/2023-03/dd183e484b824553bbba63c969042ea4.pdf"))
print(requests.get("http://www.bit.edu.cn/xww/zhxw/88672.htm").status_code)

SplitResult(scheme='http', netloc='bit.edu.cn', path='', query='', fragment='')
ParseResult(scheme='http', netloc='www.bit.edu.cn', path='/docs/2023-03/dd183e484b824553bbba63c969042ea4.pdf', params='', query='', fragment='')
404


In [33]:
# Set the main website to crawl
BASE_URL = 'http://bit.edu.cn'

# Set the maximum number of pages to crawl (optional)
MAX_PAGES = 10000

# Initialize the set of visited pages
visited_pages = set()
dead_links = set()
pdf_files = set()
# Initialize the queue of pages to crawl
pages_to_crawl = [BASE_URL]

# Create a session object for making HTTP requests
session = requests.Session()



In [34]:
# Define a function to fetch a page asynchronously
async def fetch_page(url,session):
    async with aiohttp.ClientSession() as client_session:
        async with client_session.get(url) as response:
            return await response.text()

In [35]:
# # Define a function to crawl a page and extract its links
# async def crawl_page(url, session):
#     print(f'Crawling {url}...')
#     try:
#         page_html = await fetch_page(url, session)
#         soup = BeautifulSoup(page_html, 'html.parser')
#         links = soup.find_all('a')
#         for link in links:
#             link_url = link.get('href')
#             if link_url and 'http' not in link_url:
#                 abs_url = urljoin(url, link_url)
#                 domain = urlparse(BASE_URL).netloc
#                 if domain in abs_url and abs_url not in visited_pages:
#                     visited_pages.add(abs_url)
#                     pages_to_crawl.append(abs_url)
#     except Exception as e:
#         print(f'Error crawling {url}: {e}')

In [36]:
def check_dead_links(url):
    try:
        if url in dead_links:
            return 1
        with requests.get(url) as response:
            if response.status_code >= 200 and response.status_code < 400:
                visited_pages.add(url)
                pages_to_crawl.append(url)
                return 0
            else:
                dead_links.add(url)
                return 1
    except Exception as e:
        print(f'Error checking URL {url}: {e}')
        dead_links.add(url)
        return 1

In [37]:
async def crawl_page(url, session,depth):
    print(f'Crawling {url}...')
    try:
        depth -= 1
        page_html = await fetch_page(url, session)
        soup = BeautifulSoup(page_html, 'html.parser')
        links = soup.find_all('a')
        for link in links:
            link_url = link.get('href')
            if link_url:
                if 'http' not in link_url:
                    abs_url = urljoin(url, link_url)
                else:
                    abs_url = link_url
                domain = urlparse(BASE_URL).netloc
                split_url = urlsplit(abs_url)
                if split_url.path.endswith('pdf'):
                    pdf_files.add(abs_url)
                    continue
                if domain in split_url.netloc and abs_url not in visited_pages:
                    result = check_dead_links(abs_url)
                    if not result:
                        if depth ==0:
                            break
                        else:
                           await crawl_page(abs_url,session,depth)
    except Exception as e:
        print(f'Error crawling {url}: {e}')


In [38]:
# Define a function to run the crawler asynchronously
async def run_crawler(depth):
    while len(pages_to_crawl) > 0 and len(visited_pages) < MAX_PAGES:
        url = pages_to_crawl.pop(0)
        if url not in visited_pages:
            visited_pages.add(url)
            await crawl_page(url, session,depth)


In [39]:
depth = input('depth=')
# Run the crawler asynchronously
loop = asyncio.get_event_loop()
loop.run_until_complete(run_crawler(depth))

# Print the list of visited pages
print(f'Visited {len(visited_pages)} pages:')
for page in visited_pages:
    print(page)

Crawling http://bit.edu.cn...
Crawling http://www.bit.edu.cn/xww/szllw/szw2020/index.htm...
Crawling http://www.bit.edu.cn/xww/szllw/szw2020/zyjhsz/index.htm...
Crawling http://www.bit.edu.cn/xww/szllw/szw2020/zcwjsz/index.htm...
Crawling http://www.bit.edu.cn/xww/szllw/szw2020/xyspsz/index.htm...
Crawling http://www.bit.edu.cn/xww/szllw/szw2020/wwyt/wwhdsz/index.htm...
Crawling http://www.bit.edu.cn/xww/szllw/szw2020/tgxz/index.htm...
Crawling http://www.bit.edu.cn/xww/index.htm...
Crawling http://www.bit.edu.cn/xww/zhxw/djsz/bc54cdaf680342d598c0f06009e0f05d.htm...
Crawling http://www.bit.edu.cn/xww/zhxw/djsz/8aee73d588e3486a99a61a9f47d1c9ab.htm...
Crawling http://www.bit.edu.cn/xww/zhxw/yljs/8d22c15519af48459084ee45988ed515.htm...
Crawling http://www.bit.edu.cn/xww/zhxw/djsz/572e31ec9db94c9b942c073fff98ae0c.htm...
Crawling http://www.bit.edu.cn/xww/zhxw/djsz/e92312e72a61497f97dd00dfd1b70267.htm...
Crawling http://xcb.bit.edu.cn...
Crawling http://www.bit.edu.cn/xww/zhxw/jxky1/dad53b4

In [42]:
print(len(pdf_files))
print(pdf_files)

88
{'http://kjc.bit.edu.cn/docs/2021-04/97737e44c2534a9386a2d91f2c3faea7.pdf', 'http://kjc.bit.edu.cn/docs/2021-03/fb5038f73c9f49b685177f165417c33c.pdf', 'http://kjc.bit.edu.cn/pub/kxjzyjy/docs/20191011012440090888.pdf', 'http://bit.edu.cn/docs/2023-04/27a8c230e5644fb4a4adf3d4f36e3d92.pdf', 'http://dzb.bit.edu.cn/docs/2020-09/d955e93b32754c9c8a9453e623967d13.pdf', 'http://dzb.bit.edu.cn/docs/2023-05/174ebdb75ae346ad93140062c3cdd006.pdf', 'https://dzb.bit.edu.cn/docs/2023-04/eeb61f5a904346bd8928cfc6f86e5b5c.pdf', 'https://xxgk.bit.edu.cn/docs/2023-04/f2809e0eed5040f28ec17b90710d190a.pdf', 'http://www.bit.edu.cn/docs/2023-03/dd183e484b824553bbba63c969042ea4.pdf', 'http://webpub.info.bit.edu.cn/pub/bit/docs/2013-09/20130916171822527303.pdf', 'http://bit.edu.cn/docs/2016-09/20160908074415854893.pdf', 'http://kjc.bit.edu.cn/docs/2021-03/40424781664942ddb3618641bec8d926.pdf', 'http://www.bit.edu.cn/docs/2022-10/21065d5a85e940ac907a4e6b6a00f5d8.pdf', 'http://www.bit.edu.cn/docs/2020-09/202009

In [41]:
print(len(visited_pages))
print(len(dead_links))

3652
347
